In [ ]:
import torch

## Get Data

## Specify Model

In [ ]:
aligner = Aligner()
aligner.to(device)
Aligner_optimizer = torch.optim.AdamW(aligner.parameters(), lr=0.025, betas=(0.5, 0.999))

## Choo Choo Train

In [ ]:
  aligner.train()
